In [ ]:
import argparse
import os
import shutil

from cached_path import cached_path
from FILM69.tts.f5_tts.model import CFM, UNetT, DiT, Trainer
from FILM69.tts.f5_tts.model.utils import get_tokenizer
from FILM69.tts.f5_tts.model.dataset import load_dataset
from importlib.resources import files

In [ ]:
args=dict(
    exp_name='F5TTS_Base',
    dataset_name='th',
    learning_rate=1e-05,
    batch_size_per_gpu=3200,
    batch_size_type='frame',
    max_samples=4,
    grad_accumulation_steps=1,
    max_grad_norm=1.0,
    epochs=1000,
    num_warmup_updates=0,
    save_per_updates=3*500,
    last_per_updates=3*500,
    keep_last_n_checkpoints=-1,
    finetune=True,
    pretrain=None,
    tokenizer='pinyin',
    tokenizer_path=None,
    log_samples=True,
    logger='tensorboard',
    bnb_optimizer=False
    )

In [ ]:
os.environ["path_data"]="/mnt/data/notebook/shared/FILM/research/F5/data"
os.environ["path_ckpts"]="/mnt/data/notebook/shared/FILM/research/F5/ckpts"

In [ ]:
# -------------------------- Dataset Settings --------------------------- #
target_sample_rate = 24000
n_mel_channels = 100
hop_length = 256
win_length = 1024
n_fft = 1024
mel_spec_type = "vocos"  # 'vocos' or 'bigvgan'

def main():

    checkpoint_path = f'{os.environ["path_ckpts"]}/{args["dataset_name"]}'

    # Model parameters based on experiment name
    if args["exp_name"] == "F5TTS_Base":
        wandb_resume_id = None
        model_cls = DiT
        model_cfg = dict(dim=1024, depth=22, heads=16, ff_mult=2, text_dim=512, conv_layers=4)
        if args["finetune"]:
            if args["pretrain"] is None:
                ckpt_path = str(cached_path("hf://SWivid/F5-TTS/F5TTS_Base/model_1200000.pt"))
            else:
                ckpt_path = args["pretrain"]
    elif args["exp_name"] == "E2TTS_Base":
        wandb_resume_id = None
        model_cls = UNetT
        model_cfg = dict(dim=1024, depth=24, heads=16, ff_mult=4)
        if args["finetune"]:
            if args["pretrain"] is None:
                ckpt_path = str(cached_path("hf://SWivid/E2-TTS/E2TTS_Base/model_1200000.pt"))
            else:
                ckpt_path = args["pretrain"]

    if args["finetune"]:
        if not os.path.isdir(checkpoint_path):
            os.makedirs(checkpoint_path, exist_ok=True)

        file_checkpoint = os.path.basename(ckpt_path)
        if not file_checkpoint.startswith("pretrained_"):  # Change: Add 'pretrained_' prefix to copied model
            file_checkpoint = "pretrained_" + file_checkpoint
        file_checkpoint = os.path.join(checkpoint_path, file_checkpoint)
        if not os.path.isfile(file_checkpoint):
            shutil.copy2(ckpt_path, file_checkpoint)
            print("copy checkpoint for finetune")

    # Use the tokenizer and tokenizer_path provided in the command line arguments
    tokenizer = args["tokenizer"]
    if tokenizer == "custom":
        if not args["tokenizer_path"]:
            raise ValueError("Custom tokenizer selected, but no tokenizer_path provided.")
        tokenizer_path = args["tokenizer_path"]
    else:
        tokenizer_path = args["dataset_name"]

    vocab_char_map, vocab_size = get_tokenizer(tokenizer_path, tokenizer)

    print("\nvocab : ", vocab_size)
    print("\nvocoder : ", mel_spec_type)

    mel_spec_kwargs = dict(
        n_fft=n_fft,
        hop_length=hop_length,
        win_length=win_length,
        n_mel_channels=n_mel_channels,
        target_sample_rate=target_sample_rate,
        mel_spec_type=mel_spec_type,
    )

    model = CFM(
        transformer=model_cls(**model_cfg, text_num_embeds=vocab_size, mel_dim=n_mel_channels),
        mel_spec_kwargs=mel_spec_kwargs,
        vocab_char_map=vocab_char_map,
    )

    trainer = Trainer(
        model,
        args["epochs"],
        args["learning_rate"],
        num_warmup_updates=args["num_warmup_updates"],
        save_per_updates=args["save_per_updates"],
        keep_last_n_checkpoints=args["keep_last_n_checkpoints"],
        checkpoint_path=checkpoint_path,
        batch_size=args["batch_size_per_gpu"],
        batch_size_type=args["batch_size_type"],
        max_samples=args["max_samples"],
        grad_accumulation_steps=args["grad_accumulation_steps"],
        max_grad_norm=args["max_grad_norm"],
        logger=args["logger"],
        wandb_project=args["dataset_name"],
        wandb_run_name=args["exp_name"],
        wandb_resume_id=wandb_resume_id,
        log_samples=args["log_samples"],
        last_per_updates=args["last_per_updates"],
        bnb_optimizer=args["bnb_optimizer"],
    )

    train_dataset = load_dataset(args["dataset_name"], tokenizer, mel_spec_kwargs=mel_spec_kwargs)

    trainer.train(
        train_dataset,
        resumable_with_seed=666,  # seed for shuffling dataset
    )




In [ ]:
main()